In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import requests
import os
import scipy.signal
import scipy.stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from dateutil.relativedelta import relativedelta
import time
from dateutil.tz import gettz
from datetime import timedelta
import calendar
import datetime
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import xgboost as xgb

In [2]:
def download_nrg(start_date, end_date, devid):
    
    address = "https://m3.meazon.com/"

    r = requests.post(address + "api/auth/login",
                      json={'username': 'meazon@thingsboard.org', 'password': 'meazon'}).json()

    # acc_token is the token to be used in the next request
    acc_token = 'Bearer' + ' ' + r['token']
    
    # request all descriptors that have ever been assigned to this device
    r1 = requests.get(url = address+"api/plugins/telemetry/DEVICE/"+devid+"/keys/timeseries",headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
 
    
#     descriptors = [x for x in r1]
#     descriptors = ','.join(descriptors)
    descriptors = 'pwrA,pwrB,pwrC'

    start_time = str(start_date)
    end_time = str(end_date)

    # address =  "https://m3.meazon.com/"
    r2 = requests.get(
        url=address + "api/plugins/telemetry/DEVICE/" + devid + "/values/timeseries?keys="+descriptors+"&startTs=" + start_time + "&endTs=" + end_time + "&agg=NONE&limit=1000000",
        headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()

    # download temperature


    if (r2):

        df = pd.DataFrame([])
        

        
        # read all descriptors at once
        for desc in r2.keys():
            df1 = pd.DataFrame(r2[desc])
            df1.set_index('ts', inplace=True)
            df1.columns = [str(desc)]
            df = pd.concat([df,df1], axis = 1)
        print(df.head())  
        

        df.reset_index(drop=False, inplace=True)
        df['Timestamp'] = pd.to_datetime(df['ts'], unit='ms')
        #         df['ts'] = df['ts'].dt.tz_localize('utc').dt.tz_convert('Europe/Athens').dt.tz_localize(None)
        
        df = df.sort_values(by=['Timestamp'])
        df.reset_index(drop=True, inplace=True)
        
    else:
        df = pd.DataFrame([])
        print('Empty json!')

    return df


def fill_missing_values(df, interv):
    # merge rows with same datetime to exclude nans
    df = df.groupby(df.index).max()
    df.sort_index(inplace=True)
    # create datetime series to import missing dates and reindex
    start_date = df.index[0]
    end_date = df.index[-1]
    idx = pd.date_range(start_date, end_date, freq=str(interv) + 'T')
    df = df.reindex(idx)

    return df


def create_nrg_table(df):
    df['cnrgA'] = df['cnrgA'].astype(float)
    df['cnrgB'] = df['cnrgB'].astype(float)
    df['cnrgC'] = df['cnrgC'].astype(float)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d %H:%M:%S.%f')
    df = df.sort_values(by="Timestamp")

    df['Timestamp'] = df['Timestamp'].astype('datetime64[s]')
    df = df.set_index('Timestamp', drop=True)
    df.index = df.index.map(lambda x: x.replace(second=0))

    ####################
    tmpdf = pd.DataFrame(df['cnrgA'].dropna())
    tmpdf['minutes'] = tmpdf.index.minute
    tmpdf['interv'] = tmpdf['minutes'].shift(-1) - tmpdf['minutes']
    interv = int(tmpdf['interv'].value_counts().idxmax())
    del tmpdf

    df.index = df.index.round(str(interv) + 'T')

    df = fill_missing_values(df, interv)
    df['Timestamp'] = df.index
    df['total'] = df['cnrgA'] + df['cnrgB'] + df['cnrgC']

    return [df, interv]


def conv_to_consumption(df):
    #     convert cumulative energy to consumed energy

    df['diffA'] = np.nan
    df['diffB'] = np.nan
    df['diffC'] = np.nan
    df.loc[((df.cnrgA.isna() == False) & (df.cnrgA.shift().isna() == False)), 'diffA'] = df.cnrgA - df.cnrgA.shift()
    df.loc[(df.cnrgB.isna() == False) & (df.cnrgB.shift().isna() == False), 'diffB'] = df.cnrgB - df.cnrgB.shift()
    df.loc[(df.cnrgC.isna() == False) & (df.cnrgC.shift().isna() == False), 'diffC'] = df.cnrgC - df.cnrgC.shift()

    df.diffA.iloc[0] = 0
    df.diffB.iloc[0] = 0
    df.diffC.iloc[0] = 0

    df['total'] = np.nan
    df.loc[(df.diffA.isna() == False) & (df.diffB.isna() == False) & (
            df.diffC.isna() == False), 'total'] = df.diffA + df.diffB + df.diffC

    return df


def find_nans(cnrg, energy, interv):
    # store starting points of NaNs

    df_start = energy[((energy[cnrg].isnull()) & (energy[cnrg].shift().isnull() == False))]
    if df_start.empty == False:

        if (np.isnan(energy[cnrg].iloc[0]) == True):

            df2 = pd.DataFrame([])
            df2['endpoint'] = energy.index[(energy[cnrg].isnull()) & (energy[cnrg].shift(-1).isnull() == False)].copy()
            df2 = df2.iloc[1:]

            df_start['endpoint'] = df2['endpoint']
        else:
            df_start['endpoint'] = energy.index[(energy[cnrg].isnull()) & (energy[cnrg].shift(-1).isnull() == False)]
        df_start = df_start.drop(['cnrgA', 'cnrgB', 'cnrgC', 'total'], axis=1)

        df_start['previous_dt'] = df_start.index - timedelta(minutes=interv)
        df_start['next_dt'] = df_start.endpoint + timedelta(minutes=interv)
        df_start['previous_week_start'] = df_start.previous_dt - timedelta(days=7)
        df_start['previous_week_end'] = df_start.next_dt - timedelta(days=7)

        df_start['next_week_start'] = df_start.previous_dt + timedelta(days=7)
        df_start['next_week_end'] = df_start.next_dt + timedelta(days=7)

    return df_start


def backfill(row, cnrg, energy, interv):
    tmp = pd.DataFrame(energy.loc[row.previous_week_start:row.previous_week_end])
    if tmp.shape[0] == 0:
        tmp = pd.DataFrame(
            energy.loc[row.previous_week_start + timedelta(days=7):row.previous_week_end + timedelta(days=7)])
    if tmp.shape[0] > 0:

        start1 = tmp[cnrg].iloc[0]  # starting point of previous week
        start2 = energy[cnrg].loc[row.previous_dt]  # starting point of current week

        diff1 = tmp[cnrg].iloc[-1] - start1  # diff of previous week
        diff2 = energy[cnrg].loc[row.next_dt] - start2  # diff of current week

        for i in range(1, tmp.shape[0] - 1):
            if ((tmp[cnrg].iloc[i] - start1) != diff1 and (diff1 != 0)):
                perc = (tmp[cnrg].iloc[i] - start1) / diff1
                energy[cnrg].loc[row.previous_dt + timedelta(minutes=i * interv)] = perc * diff2 + start2

            else:
                perc = 0
                energy[cnrg].loc[row.previous_dt + timedelta(minutes=i * interv)] = energy[cnrg].loc[
                    row.previous_dt + timedelta(minutes=i * interv - interv)]


def forwardfill(row, cnrg, energy, interv):
    tmp = energy.loc[row.next_week_start:row.next_week_end]
    if tmp.shape[0] > 0:
        k = 1
        # while all intermediate values are nan
        while ((tmp[cnrg].iloc[0] + tmp[cnrg].iloc[-1] == tmp[cnrg].sum()) & (
                row.next_week_start - timedelta(days=k) >= energy.Timestamp.iloc[
            0])):
            tmp = energy.loc[row.next_week_start - timedelta(days=k):row.next_week_end - timedelta(days=k)]
            k = k + 1

        if tmp.shape[0] > 0:
            start1 = tmp[cnrg].iloc[0]  # starting point of next week
            diff1 = tmp[cnrg].iloc[-1] - start1  # diff of next week

            start2 = energy[cnrg].loc[row.previous_dt]  # starting point of current week
            diff2 = energy[cnrg].loc[row.next_dt] - start2  # diff of current week

            for i in range(1, tmp.shape[0] - 1):

                # if ((tmp[cnrg].iloc[i] - start1) != diff1 and (diff1 != 0) and (np.isnan(start1)==False)):
                if ((tmp[cnrg].iloc[i] - start1) != diff1 and (diff1 != 0)):

                    perc = (tmp[cnrg].iloc[i] - start1) / diff1
                    energy[cnrg].loc[row.previous_dt + timedelta(minutes=i * interv)] = perc * diff2 + start2

                else:
                    perc = 0
                    energy[cnrg].loc[row.previous_dt + timedelta(minutes=i * interv)] = energy[cnrg].loc[
                        row.previous_dt + timedelta(minutes=i * interv - interv)]

        else:
            energy[cnrg] = energy[cnrg].interpolate(method='linear')


    else:
        k = 1
        while ((tmp.shape[0] == 0) & ((row.next_dt + timedelta(hours=24 * k)) <= energy.Timestamp.iloc[-1])):
            tmp = energy.loc[row.previous_dt + timedelta(hours=24 * k):row.next_dt + timedelta(hours=24 * k)]

            k = k + 1

        if tmp.shape[0] > 0:
            start1 = tmp[cnrg].iloc[0]  # starting point of next week
            diff1 = tmp[cnrg].iloc[-1] - start1  # diff of next week

            start2 = energy[cnrg].loc[row.previous_dt]  # starting point of current week
            diff2 = energy[cnrg].loc[row.next_dt] - start2  # diff of current week

            for i in range(1, tmp.shape[0] - 1):

                # if ((tmp[cnrg].iloc[i] - start1) != diff1 and (diff1 != 0) and (np.isnan(start1)==False)):
                if ((tmp[cnrg].iloc[i] - start1) != diff1 and (diff1 != 0)):
                    perc = (tmp[cnrg].iloc[i] - start1) / diff1
                    energy[cnrg].loc[row.previous_dt + timedelta(minutes=i * interv)] = perc * diff2 + start2
                else:
                    perc = 0
                    energy[cnrg].loc[row.previous_dt + timedelta(minutes=i * interv)] = energy[cnrg].loc[
                        row.previous_dt + timedelta(minutes=i * interv - interv)]
        else:
            energy[cnrg] = energy[cnrg].interpolate(method='linear')




def fill_nans(nrg, energy, interv):
    for cnrg in nrg:

        df_start = find_nans(cnrg, energy, interv)
        prev_status = df_start.shape[0]
        df_start.apply((lambda x: backfill(x, cnrg, energy, interv)), axis=1)
        print('backfill ended')

        while energy[cnrg].isna().sum() > 0:
            
            if ((energy[cnrg].isna().sum() == 1) & np.isnan(energy[cnrg].iloc[0])):
                energy[cnrg].iloc[0] = energy[cnrg].iloc[1]

            else:
                df_start = find_nans(cnrg, energy, interv)
                cur_status = df_start.shape[0]
                if ((df_start.empty == False) and (cur_status < prev_status)):
                    df_start.apply((lambda x: forwardfill(x, cnrg, energy, interv)), axis=1)

                else:
                    
                    energy[cnrg] = energy[cnrg].interpolate(method='linear',limit_direction ='both')
                    
                prev_status = cur_status
        print('forward fill ended')


def fill_dropped_nrg(df, nrg, interv):
    for cnrg in nrg:
        dfnew = df[np.isfinite(df[cnrg])].copy()
        dropped = dfnew[dfnew[cnrg] < dfnew[cnrg].shift()]

        if dropped.empty == False:
            # keep endpoint of range of reseted values
            dropped['endpoint1'] = dfnew.index[dfnew[cnrg] > dfnew[cnrg].shift(-1)]

            # shift endpoints to match starting points and set last endpoint as the last instance of df
            dropped['endpoint'] = dropped['endpoint1'].shift(-1)
            dropped['endpoint'].iloc[-1] = df.index[-1]

            dropped.apply((lambda x: correct_dropped(x, cnrg, df, interv)), axis=1)

    return df


def correct_dropped(row, cnrg, df, interv):
    # df[cnrg].loc[row.Timestamp:row.endpoint] = np.sum([df[cnrg], df[cnrg].loc[row.endpoint1]])
    if df[cnrg].loc[row.Timestamp] > df[cnrg].loc[row.endpoint1 - timedelta(minutes=interv)]:
        df[cnrg].loc[row.endpoint1] = df[cnrg].loc[row.Timestamp]
    else:
        df[cnrg].loc[row.Timestamp:row.endpoint] = np.sum(
            [df[cnrg], np.abs(df[cnrg].loc[row.endpoint1] - df[cnrg].loc[row.Timestamp])])




def get_energy_data(start_date, end_date, devid):
    print('Downloading cumulative energy...')
    dfcnrg = download_nrg(start_date, end_date, devid)
    if dfcnrg.empty == True:
        energy = pd.DataFrame([])
        interv = np.nan
        return energy
    else:
        return dfcnrg
#         nrg = ['cnrgA', 'cnrgB', 'cnrgC']
#         [energy, interv] = create_nrg_table(dfcnrg)

#         if ((energy.cnrgA.isna().sum() > 0.4 * energy.shape[0]) | (energy.shape[0] < 7 * 24 * (60 / interv))):
#             print('Very few values!')
#             energy = pd.DataFrame([])
#             interv = np.nan
#             return energy
#         else:

#             print('Correcting energy dropdowns')
#             energy = fill_dropped_nrg(energy, nrg, interv)

#             print('Filling missing values')
#             energy = conv_to_consumption(energy)
#             thresA = energy.diffA.mean() + 3 * energy.diffA.std()
#             thresB = energy.diffB.mean() + 3 * energy.diffB.std()
#             thresC = energy.diffC.mean() + 3 * energy.diffC.std()

#             energy.loc[(energy.diffA.shift(-1) > thresA) & (energy.cnrgA.shift(-2) == np.nan), 'cnrgA'] = np.nan
#             energy.loc[(energy.diffB.shift(-1) > thresB) & (energy.cnrgB.shift(-2) == np.nan), 'cnrgB'] = np.nan
#             energy.loc[(energy.diffC.shift(-1) > thresC) & (energy.cnrgC.shift(-2) == np.nan), 'cnrgC'] = np.nan

#             energy = conv_to_consumption(energy)
#             fill_nans(nrg, energy, interv)
#             energy = conv_to_consumption(energy)
            
    
#             energy.sort_values(by=['Timestamp'], inplace=True)
#             #             energy.nrgA = energy.nrgA / 1000
#             #             energy.nrgB = energy.nrgB / 1000
#             #             energy.nrgC = energy.nrgC / 1000
#             energy.set_index(pd.to_datetime(energy.Timestamp, unit='ms'), inplace=True, drop=True)


#             print('Energy df has successfully been created')
#             return energy



In [3]:
def build_model(X_train,y_train, X_test, y_test):
    
    
    def objective(parameters, n_folds = 10):
        
        # fit model to the first half set
        Dtrain = xgb.DMatrix(X_train, label=y_train)
        mdl = xgb.train(parameters, Dtrain)
        
        
        Observed = np.array([])
        Forecasted = np.array([])
        

        test_set = xgb.DMatrix(X_test, label=y_test)
        y_pred = mdl.predict(test_set)

            
        loss = 100 - appliance_acc(y_test,y_pred)
        
        print('loss:',loss)
        return {'loss': loss, 'parameters': parameters, 'status': STATUS_OK}


    # Define the space to search for best hyperparameters
    space = {
        'n_estimators': hp.quniform('n_estimators', 100, 1000, 2),
        'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
        'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'objective': 'reg:linear',
        'eval_metric':'mae',
        'booster': 'gbtree',
        'tree_method': 'exact',
        'silent': 1,
        'seed':315469
    }
    


    trials = Trials()
    
    best = fmin(fn = objective, space = space,algo = tpe.suggest,trials = trials,max_evals = 30,verbose=0)
    best_trial = sorted(trials.results, key = lambda x: x['loss'])
    params = best_trial[0]['parameters']
#     train_set = xgb.DMatrix(trainset[final_set], label = trainset['Consumption'])
#     best_bayes_model = xgb.train(params,train_set)
    xgb_acc = 100-best_trial[0]['loss']

    

#     print('Best acc:',xgb_acc)
#     mdl = best_bayes_model
        
    
    return (xgb_acc)

In [4]:
devid = '306920b0-3a42-11ea-8762-6bf954fc5af1'


end_ = (datetime.datetime.now())
       
start_ = end_ + relativedelta(months=-1)

end_time = int(end_.timestamp()) * 1000
start_time = int(start_.timestamp()) * 1000



        
# make request
indata = get_energy_data(start_time, end_time, devid)

                        pwrA           pwrB           pwrC
ts                                                        
1585324369772  1154.98510742  3.44256687164            NaN
1585324369773            NaN            NaN  3.49745583534
1585324429800  1153.28637695  3.44549489021  3.45500946045
1585324489766  1151.36450195  3.41988015175  3.46013116837
1585324550028  1149.54541016  3.49965167046  3.46891331673


In [5]:
indata.head()

,ts,pwrA,pwrB,pwrC,Timestamp
0,1585324369772,1154.98510742,3.44256687164,NaN,2020-03-27 15:52:49.772
1,1585324369773,NaN,NaN,3.49745583534,2020-03-27 15:52:49.773
2,1585324429800,1153.28637695,3.44549489021,3.45500946045,2020-03-27 15:53:49.800
3,1585324489766,1151.36450195,3.41988015175,3.46013116837,2020-03-27 15:54:49.766
4,1585324550028,1149.54541016,3.49965167046,3.46891331673,2020-03-27 15:55:50.028


In [ ]:
def window(a, w , o , copy = False):
    sh = (a.size - w + 1, w)
    st = a.strides * 2
    view = np.lib.stride_tricks.as_strided(a, strides = st, shape = sh)[0::o]
    if copy:
        return view.copy()
    else:
        return view

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def appliance_acc(y_true,y_pred):
    acc = 1-(np.sum(np.abs(y_true-y_pred))/(2*np.sum(np.abs(y_true))))
    acc = acc*100
    return acc


In [ ]:
fig = plt.figure(figsize = [17,10])
plt.grid(True)
plt.plot(sm.pwrtotal)
# plt.plot(plug1.pwr)
# plt.plot(plug2.pwr)
# plt.plot(plug4.pwr)
# plt.plot(plug5.pwr)
plt.plot(plg.pwr)
# plt.plot(plug7.pwr)
plt.legend(['Total','Freezer'])


In [ ]:
# Convert dfs to numpy arrays 
met_cur = sm['cur']
met_cur = met_cur.to_numpy()
sm = sm['pwrtotal']
sm = sm.to_numpy()


# Pass signal through median filter with 5 samples
sm = sp.signal.medfilt(sm,5)

plg = plg['pwr']
plg = plg.to_numpy()

#split to time windows with 50% overlap
meter = window(sm, 10,5)
met_cur = window(met_cur,10,5)
plug = window(plg,10,5)

In [ ]:
# Extract features



Y = np.array([])
X = np.empty((0,10))

# mean,min,max,median,rms,perc25,perc75,std,energy
for i in range(0,len(plug)):
    Y = np.append(Y,np.mean(plug[i,:]))
    X = np.append(X,np.array([[np.mean(meter[i,:]),np.min(meter[i,:]),np.max(meter[i,:]),\
            np.median(meter[i,:]),np.sqrt(np.mean(meter[i,:]**2, axis=None)),\
            np.percentile(meter[i,:],25),np.percentile(meter[i,:],75),np.std(meter[i,:]),\
                                    np.sum(meter[i,:]),np.mean(met_cur[i,:])]]),axis = 0)
    
    

In [ ]:
# n_test = int(0.33*len(features))
# trainX, testX = features[0:-n_test], features[-n_test:]
# trainY, testY = appl[0:-n_test], appl[-n_test:]

# split train and test

trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.33, random_state=42,shuffle = False)
print('trainX size:',trainX.shape)
print('trainY size:',trainY.shape)
acc = build_model(trainX,trainY, testX, testY)

In [ ]:
kf = KFold(n_splits=10)
kf.get_n_splits(X)
err = []

for train_index, test_index in kf.split(X):
    trainX, testX = X[train_index], X[test_index]
    trainY, testY = Y[train_index],Y[test_index]
    
    acc = build_model(trainX,trainY, testX, testY)
    
    print('Accuracy of fold is:', acc)
    err.append(acc)

print('Mean accuracy of 10 folds:',np.mean(err))

In [ ]:
kf = KFold(n_splits=10)
kf.get_n_splits(X)
err = []

for train_index, test_index in kf.split(X):
    trainX, testX = X[train_index], X[test_index]
    trainY, testY = Y[train_index],Y[test_index]
    
    mdl = RandomForestRegressor(n_estimators=8,random_state=0)
    mdl.fit(trainX, trainY)
    pred = mdl.predict(testX)
    
    acc = appliance_acc(testY,pred)
    
    print('Accuracy of fold %d is: %f' % (i, acc))
    err.append(acc)

print('Mean accuracy of 10 folds:',np.mean(err))

In [ ]:
mdl = RandomForestRegressor(n_estimators=32,random_state=0)
# mdl = KNeighborsRegressor(n_neighbors=5)
mdl.fit(trainX, trainY)
pred = mdl.predict(testX)

In [ ]:
fig = plt.figure(figsize = [17,10])
plt.grid(True)
plt.plot(testY)
plt.plot(pred)

In [ ]:
acc = appliance_acc(testY,pred)
print('Accuracy:',acc)

In [ ]:


mape = mean_absolute_percentage_error(testY,pred)
